In [1]:
import os
import glob
import tqdm
import numpy as np
import pandas as pd
import scipy.io as sio
import tensorflow as tf
import plotly.express as px
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from sklearn.utils.class_weight import compute_class_weight


import os
import sys
import glob
import h5py
import numpy as np
from torch.utils.data import Dataset


In [ ]:

def download():
    BASE_DIR = './'
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    if not os.path.exists(DATA_DIR):
        os.mkdir(DATA_DIR)
    if not os.path.exists(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048')):
        www = 'https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip'
        zipfile = os.path.basename(www)
        os.system('wget --no-check-certificate %s; unzip %s' % (www, zipfile))
        os.system('mv %s %s' % (zipfile[:-4], DATA_DIR))
        os.system('rm %s' % (zipfile))


def load_data(partition):
    download()
    BASE_DIR = './'
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    all_data = []
    all_label = []
    for h5_name in glob.glob(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048', 'ply_data_%s*.h5'%partition)):
        f = h5py.File(h5_name)
        data = f['data'][:].astype('float32')
        label = f['label'][:].astype('int64')
        f.close()
        all_data.append(data)
        all_label.append(label)
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    return all_data, all_label


def load_scanobjectnn_data(partition):
    BASE_DIR = './'
    DATA_DIR = os.path.join(BASE_DIR, 'data')
    all_data = []
    all_label = []

    h5_name = BASE_DIR + '/data/' + partition + '_objectdataset_augmentedrot_scale75.h5'
    f = h5py.File(h5_name)
    data = f['data'][:].astype('float32')
    label = f['label'][:].astype('int64')
    f.close()
    all_data.append(data)
    all_label.append(label)
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    return all_data, all_label


def translate_pointcloud(pointcloud):
    xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
    xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
       
    translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')
    return translated_pointcloud


def jitter_pointcloud(pointcloud, sigma=0.01, clip=0.02):
    N, C = pointcloud.shape
    pointcloud += np.clip(sigma * np.random.randn(N, C), -1*clip, clip)
    return pointcloud


class ModelNet40(Dataset):
    def __init__(self, num_points, partition='train'):
        self.data, self.label = load_data(partition)
        self.num_points = num_points
        self.partition = partition        

    def __getitem__(self, item):
        pointcloud = self.data[item][:self.num_points]
        label = self.label[item]
        if self.partition == 'train':
            pointcloud = translate_pointcloud(pointcloud)
            np.random.shuffle(pointcloud)
        return pointcloud, label

    def __len__(self):
        return self.data.shape[0]


class ScanObjectNN(Dataset):
    def __init__(self, num_points, partition='training'):
        self.data, self.label = load_scanobjectnn_data(partition)
        self.num_points = num_points
        self.partition = partition        

    def __getitem__(self, item):
        pointcloud = self.data[item][:self.num_points]
        label = self.label[item]
        if self.partition == 'training':
            pointcloud = translate_pointcloud(pointcloud)
            np.random.shuffle(pointcloud)
        return pointcloud, label

    def __len__(self):
        return self.data.shape[0]




train=ModelNet40(partition='train', num_points=1024)

test=ModelNet40(partition='test', num_points=1024)

X_train=train.data[:,:128,:]
X_test=test.data[:,:128,:]
y_train=train.label
y_test=test.label

y_train=np.eye(40)[y_train[:,0]]
y_test=np.eye(40)[y_test[:,0]]




#l=X_train.shape[0]
#ch=np.random.choice(l,int(l/1))
#X_train=X_train[ch]
#y_train=y_train[ch]

In [ ]:
## DECOMPOSED LAPLACIAN
print("LOADING MODEL")
def conv_bn(x, filters,):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid",kernel_regularizer=tf.keras.regularizers.l2( l=0.000005))(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

def conv2_bn(x, filters,):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid",kernel_regularizer=tf.keras.regularizers.l2( l=0.000005))(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return x

def dense_bn(x, filters):
    x = layers.Dense(filters,kernel_regularizer=tf.keras.regularizers.l2( l=0.000005))(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)



def matrix_block(inp,matrix,output_shape=32):
  x1=conv_bn(inp,output_shape)
  x2=K.batch_dot(matrix,x1,[2,1])             
  x2 = tf.keras.layers.BatchNormalization(momentum=0.0)(x2)
  return tf.keras.layers.concatenate([x1,x2],axis=-1)

class simplify_e(tf.keras.layers.Layer): 
   def __init__(self,n=32,**kwargs): 
      super(simplify_e, self).__init__(**kwargs)
      self.n=n
   def call(self, e):
      bs=K.shape(e)[0]
      first_shape=e.shape[1]
      mask1=tf.ones((bs,self.n))
      mask2=tf.zeros((bs,128-self.n))
      mask=tf.concat([mask1,mask2],axis=1)
      out=e*mask
      return out

inp_spatial=tf.keras.layers.Input((X_train.shape[1],3))

x=conv_bn(inp_spatial,8)

out=tf.sqrt(tf.reduce_sum((tf.expand_dims(x, 2)-tf.expand_dims(x, 1))**2,axis=3))

mask_out=tf.linalg.diag(np.ones((1,128)).astype(np.float32))*1
out=out*(1-mask_out)
max_=tf.math.reduce_max(out,axis=-1,keepdims=True)
max_=tf.math.reduce_max(max_,axis=-2,keepdims=True)
out=max_-out
D = tf.reduce_sum(out , axis = 2)
D_sqrt = tf.divide(1.0 , tf.sqrt(D))
D_sqrt = tf.linalg.diag(D_sqrt)

I = tf.ones_like(D , dtype = tf.float32)
I = tf.linalg.diag( I )
out = I - K.batch_dot(D_sqrt , K.batch_dot(out , D_sqrt,[2,1]),[2,1])


e,v=tf.linalg.eigh(
    out, name=None
)
e2=simplify_e()(e)

e2=tf.linalg.diag(e2)
left=K.batch_dot(v,e2,[2,1])
matrix_simp=K.batch_dot(left,tf.transpose(v,[0,2,1]),[2,1])


x1=matrix_block(inp_spatial,matrix_simp,32)
x2=matrix_block(x1,matrix_simp,128)
x_all=conv_bn(x2,256)
flatten=tf.keras.layers.GlobalMaxPool1D()(x_all)
dp=tf.keras.layers.Dropout(0.5)(flatten)



outputs = layers.Dense(y_train.shape[-1], activation="softmax")(dp)

model = keras.Model(inputs=inp_spatial, outputs=outputs, name="pointnet")
model.summary()
BATCH_SIZE = 64
model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["categorical_accuracy"],
)
checkpoint_filepath = 'weights/bl1024v1_1_2.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_categorical_accuracy',
    mode='max',
    save_best_only=True)

def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * tf.math.exp(-0.003)
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

class_weight = dict(zip(np.unique(y_train.argmax(axis=-1)),compute_class_weight(class_weight = "balanced", classes= np.unique(y_train.argmax(axis=-1)), y= y_train.argmax(axis=-1))))

model.fit(X_train,y_train, epochs=2000, batch_size=BATCH_SIZE, validation_data=(X_test,y_test),callbacks=[model_checkpoint_callback,lr_callback],class_weight = class_weight)


